In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from base_tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier as SKDecisionTreeClassifier
from sklearn.tree import plot_tree
pd.set_option('display.max_colwidth', 10)
pd.set_option("display.max_columns", None)

### Testing for CSIC-Web-2010 data

In [40]:
# Load the iris dataset
data_table = pd.read_csv('./dataset/csic_database.csv')
data_table = data_table.drop(columns=['classification'])
# data_table = pd.read_csv('./dataset/KDDCup99.csv')
# data_table.head()

In [41]:
data_table['lenght'].unique()

array([nan, 'Content-Length: 68', 'Content-Length: 63',
       'Content-Length: 4', 'Content-Length: 33', 'Content-Length: 43',
       'Content-Length: 232', 'Content-Length: 17', 'Content-Length: 234',
       'Content-Length: 72', 'Content-Length: 66', 'Content-Length: 42',
       'Content-Length: 231', 'Content-Length: 241', 'Content-Length: 71',
       'Content-Length: 59', 'Content-Length: 240', 'Content-Length: 276',
       'Content-Length: 67', 'Content-Length: 38', 'Content-Length: 267',
       'Content-Length: 268', 'Content-Length: 74', 'Content-Length: 62',
       'Content-Length: 37', 'Content-Length: 242', 'Content-Length: 266',
       'Content-Length: 64', 'Content-Length: 248', 'Content-Length: 239',
       'Content-Length: 271', 'Content-Length: 253',
       'Content-Length: 251', 'Content-Length: 60', 'Content-Length: 246',
       'Content-Length: 237', 'Content-Length: 262',
       'Content-Length: 243', 'Content-Length: 65', 'Content-Length: 259',
       'Content-Leng

In [35]:
data_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61065 entries, 0 to 61064
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       61065 non-null  object 
 1   Method           61065 non-null  object 
 2   User-Agent       61065 non-null  object 
 3   Pragma           61065 non-null  object 
 4   Cache-Control    61065 non-null  object 
 5   Accept           60668 non-null  object 
 6   Accept-encoding  61065 non-null  object 
 7   Accept-charset   61065 non-null  object 
 8   language         61065 non-null  object 
 9   host             61065 non-null  object 
 10  cookie           61065 non-null  object 
 11  content-type     17977 non-null  object 
 12  connection       61065 non-null  object 
 13  lenght           17977 non-null  float64
 14  content          17977 non-null  object 
 15  URL              61065 non-null  object 
dtypes: float64(1), object(15)
memory usage: 7.5+ MB


In [3]:
data_table.head()

,Unnamed: 0,Method,User-Agent,Pragma,Cache-Control,Accept,Accept-encoding,Accept-charset,language,host,cookie,content-type,connection,lenght,content,URL
0,Normal,GET,Mozill...,no-cache,no-cache,text/x...,x-gzip...,"utf-8,...",en,localh...,JSESSI...,NaN,close,NaN,NaN,http:/...
1,Normal,GET,Mozill...,no-cache,no-cache,text/x...,x-gzip...,"utf-8,...",en,localh...,JSESSI...,NaN,close,NaN,NaN,http:/...
2,Normal,POST,Mozill...,no-cache,no-cache,text/x...,x-gzip...,"utf-8,...",en,localh...,JSESSI...,applic...,Connec...,Conten...,id=3&n...,http:/...
3,Normal,GET,Mozill...,no-cache,no-cache,text/x...,x-gzip...,"utf-8,...",en,localh...,JSESSI...,NaN,close,NaN,NaN,http:/...
4,Normal,POST,Mozill...,no-cache,no-cache,text/x...,x-gzip...,"utf-8,...",en,localh...,JSESSI...,applic...,Connec...,Conten...,modo=e...,http:/...


In [6]:
# for CSIC
def get_number_from_string(x):
    if type(x) is float or type(x) is int:
        return x
    else:
        return float(x.split(': ')[-1])
#get the number in a string
data_table['lenght'] = data_table['lenght'].apply(get_number_from_string)

In [32]:
data_table.to_csv('./dataset/csic_database_preprocessed.csv', index=False)

In [ ]:
#For KDD
def sample_rows(table, column, value, n):
    # Filter rows where column 'c' equals value 'x'
    filtered_rows = table.loc[table[column] == value]
    # Sample rows where column 'c' equals value 'x' (e.g., sampling 5 rows)
    sampled_rows = filtered_rows.sample(n=n, random_state=42)
    return sampled_rows

In [ ]:
rows1 = sample_rows(data_table, 'label', 'normal', 1000)
rows2 = sample_rows(data_table, 'label', 'back', 1000)
rows3 = sample_rows(data_table, 'label', 'satan', 1000)
rows4 = sample_rows(data_table, 'label', 'warezclient', 1000)
data_table = pd.concat([rows1, rows2, rows3, rows4])

In [ ]:
data_table.describe(include='all')

In [ ]:
#remove the column with values are unique=1 or unique=all
def remove_unique_columns(df):
    vaild_cols = []
    for col in df.columns:
        if len(df[col].unique()) > 1 and len(df[col].unique()) < len(df):
            vaild_cols.append(col)
    # Return a DataFrame with only non-unique columns
    return df[vaild_cols]

In [ ]:
data_table_ = remove_unique_columns(data_table)

In [ ]:
data_table_.shape

In [ ]:
# # for CSIC
# def get_number_from_string(x):
#     if type(x) is float or type(x) is int:
#         return x
#     else:
#         return float(x.split(': ')[-1])
# #get the number in a string
# data_table_['lenght'] = data_table_['lenght'].apply(get_number_from_string)

In [ ]:
def fill_with_average(df):
    # Iterate over each column in the DataFrame
    for column in df.columns:
        # Check if the column is numeric
        if pd.api.types.is_numeric_dtype(df[column]):
            # Calculate the mean of the column, ignoring NaN values
            mean_value = df[column].mean()
            # Fill NaN values with the mean and assign back to the column
            df[column] = df[column].fillna(mean_value)
    return df

def fill_string_nan_with_none(df):
    # Iterate over each column in the DataFrame
    for column in df.columns:
        # Check if the column is of object type (usually string columns)
        if df[column].dtype == 'object' or df[column].dtype == 'O':
            # Fill NaN values with 'None'
            df[column] = df[column].fillna('None')
    return df

In [ ]:
#fill nan with average or 'None'
data_table_ = fill_with_average(data_table_)
data_table_ = fill_string_nan_with_none(data_table_)

In [ ]:
def onehot_table(df, exclude_columns=None):
    # Identify string-type columns
    string_cols = df.select_dtypes(include=['object']).columns
    string_cols_ = []
    for col in string_cols:
        if exclude_columns:
            if col in exclude_columns:
                continue
        string_cols_.append(col)
    # Create one-hot encoded DataFrame for string columns
    one_hot_encoded_df = pd.get_dummies(df[string_cols_], drop_first=False, prefix_sep='_is_')
    # Concatenate the one-hot encoded columns with the original DataFrame (excluding original string columns)
    df_final = pd.concat([df.drop(columns=string_cols_), one_hot_encoded_df], axis=1)
    return df_final

In [ ]:
#onehot exclude time series data
#For CSIC dataset
#data_table_ = onehot_table(data_table_, exclude_columns=['Unnamed: 0', 'content', 'URL'])
#For Breast dataset
#data_table_ = onehot_table(data_table_, exclude_columns=['Status'])
#For Econic dataset
#data_table_ = onehot_table(data_table_, exclude_columns=['Bankrupt?'])
#For economic
data_table_ = onehot_table(data_table_, exclude_columns=['label'])

In [ ]:
data_table_.info()

### split train and test

In [ ]:
train_table, test_table = train_test_split(data_table_, test_size=0.5)

### split the time series column

In [ ]:
## for CSIC DATASET
# train_series,test_series = train_table[['content', 'URL']],test_table[['content', 'URL']]
# train_x = train_table.drop(columns=['Unnamed: 0', 'content', 'URL']).to_numpy()
# train_y = train_table['Unnamed: 0'].apply(lambda x: 1 if x=='Anomalous' else 0).to_numpy()
# test_x = test_table.drop(columns=['Unnamed: 0', 'content', 'URL']).to_numpy()
# test_y = test_table['Unnamed: 0'].apply(lambda x: 1 if x=='Anomalous' else 0).to_numpy()
## for breast dataset
# train_x = train_table.drop(columns=['Status']).to_numpy()
# train_y = train_table['Status'].apply(lambda x: 1 if x=='Alive' else 0).to_numpy()
# test_x = test_table.drop(columns=['Status']).to_numpy()
# test_y = test_table['Status'].apply(lambda x: 1 if x=='Alive' else 0).to_numpy()
# ## for company bankruptcy
# train_x = train_table.drop(columns=['Bankrupt?']).to_numpy()
# train_y = train_table['Bankrupt?'].to_numpy()
# test_x = test_table.drop(columns=['Bankrupt?']).to_numpy()
# test_y = test_table['Bankrupt?'].to_numpy()
## for glass
# mapping = {1:0, 2:1, 3:2, 5:3, 6:4, 7:5}
# train_x = train_table.drop(columns=['Type']).to_numpy()
# train_y = train_table['Type'].apply(lambda x: mapping[x]).to_numpy()
# test_x = test_table.drop(columns=['Type']).to_numpy()
# test_y = test_table['Type'].apply(lambda x: mapping[x]).to_numpy()
##for KDD
mapping = {
    'normal':0, 
    'back':1, #dos
    'satan':2, #probe
    'warezclient':3 #r2l
}
train_x = train_table.drop(columns=['label']).to_numpy()
train_y = train_table['label'].apply(lambda x: mapping[x]).to_numpy()
test_x = test_table.drop(columns=['label']).to_numpy()
test_y = test_table['label'].apply(lambda x: mapping[x]).to_numpy()

In [ ]:
import imp
import base_tree
imp.reload(base_tree)
from base_tree import DecisionTreeClassifier

In [ ]:
data_table.describe(include='all')

In [ ]:
data_table['src_bytes'].max()

In [ ]:
# Create a Decision Tree classifier
feature_names = train_table.drop(columns=['label']).columns
class_names = list(mapping.keys())
clf = DecisionTreeClassifier(
    criterion='gini', 
    random_state=42, 
    build_method='bfs',
    max_depth=7, 
    feature_name=feature_names,
    class_name=class_names
)
# Fit the classifier to the training data
clf.fit(train_x, train_y)
# Print the accuracy of the model
accuracy = clf.score(test_x, test_y)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
from visual import print_tree
print_tree(
    tree=clf,
    feature_names=feature_names,
    class_names=class_names,
    is_classifier=True,
    figsize = (16, 12),
    fontsize = 10,
    dpi=200
)

In [ ]:
clf._n_features

In [ ]:
data = load_breast_cancer()
train_x, test_x, train_y, test_y = train_test_split(data.data, data.target, test_size=0.5)

In [ ]:
pd.DataFrame(data['data'], columns=data.feature_names).describe(include='all')

In [ ]:
data.feature_names[7]

In [ ]:
data.target_names

In [ ]:
import imp
import base_tree
imp.reload(base_tree)
from base_tree import DecisionTreeClassifier

In [ ]:
# Create a Decision Tree classifier
clf = DecisionTreeClassifier(
    criterion='gini', 
    random_state=42, 
    max_depth=6,
    feature_name=data.feature_names,
    class_name=data.target_names
)
# Fit the classifier to the training data
clf.fit(train_x, train_y)
# Print the accuracy of the model
accuracy = clf.score(test_x, test_y)
print(f"Accuracy: {accuracy:.2f}")

In [ ]:
# _, ax = plt.subplots(figsize = (16, 16), dpi=200)
# plot_tree(
#     clf,
#     feature_names=data.feature_names,
#     class_names=data.target_names,
#     filled=True,
#     ax = ax
# )

In [ ]:
from visual import print_tree

In [ ]:
print_tree(
    tree=clf,
    feature_names=data.feature_names,
    class_names=data.target_names,
    is_classifier=True,
    figsize = (16, 12),
    fontsize = 10,
    dpi=200
)

In [ ]:
import imp
import prompt_adapter
imp.reload(prompt_adapter)
from prompt_adapter import _get_condition,_get_query,get_options
from prompt_adapter import get_explanation_prompt, get_explanation_result, get_selection_prompt, get_selection_result

In [ ]:
query = _get_query(
    feature='Al Aluminum',
    val_range=(0.1, 0.5),
    det='<',
    val=0.25,
    condition={'possibility':97.3, 'result':'building_windows_non_float_processed'}
)
print(query)

In [ ]:
desc='This is a Glass Identification Data Set from UCI. It contains 10 attributes with unit measurement expect RI that weight \
percent in corresponding oxide. The response is glass type containing building_windows_float_processed, \
building_windows_non_float_processed, vehicle_windows_float_processed, containers, tableware and headlamps.'
role='doctor'
query={
    'feature':'Al Aluminum',
    'val_range':(0.1, 0.5),
    'det':'<',
    'val':0.25,
    'condition':{'possibility':97.3, 'result':'building_windows_non_float_processed'}
}
premise='Ba < 0.335, which indicates that the glass has a low barium content, suggesting it is less likely to be a type of glass \
that requires a high density or high refractive index, such as certain types of optical glass, and Mg > 2.78, which suggests \
that the glass may be designed for improved chemical durability and resistance to weathering. Magnesium is not typically present \
in high amounts in standard soda-lime glass, which is commonly used in windows. Those glass of above attributes are \
likely to be non-float processed.'
print(get_explanation_prompt(
    desc=desc,
    role=role,
    query=query,
    premise=premise
))

In [ ]:
response = 'Iris Setosa is typically distinguished by its smaller petal width, usually less than 0.8 cm, \
compared to Iris Versicolor and Iris Virginica. This characteristic likely results from evolutionary adaptations \
to specific ecological niches or pollination strategies. As such, a petal width below 0.8 cm is a strong indicator of \
Iris Setosa, aiding botanists in species classification.'
status,res = get_explanation_result(response)
print(status, res)

In [ ]:
opts = [
    'RI refractive index, range from 1.51 to 1.53, if it is less than 1.517, if has 73% possibility to be building_windows_non_float_processed.',
    'Ca Calcium, range from 5.4 to 16.2, if it is greater than 8.235, it has 50% possibility to be building_windows_float_processed.',
    'Al Aluminum, range from 0.29 to 3.5, if it is greater than 1.42, it has 70% possibility to be building_windows_non_float_processed.',
    'K Potassium, range from 0.0 to 6.21, if it is less than 0.03, it has 70% possibility to be tableware.'
]
opt_dict,opt_text = get_options(opts)
print(opt_dict)
print(opt_text)

In [ ]:
desc='This is a Glass Identification Data Set from UCI. It contains 10 attributes with unit measurement expect RI that weight \
percent in corresponding oxide. The response is glass type containing building_windows_float_processed, \
building_windows_non_float_processed, vehicle_windows_float_processed, containers, tableware and headlamps.'
role='doctor'
query={
    'feature':'Al Aluminum',
    'val_range':(0.1, 0.5),
    'det':'<',
    'val':0.25,
    'condition':{'possibility':97.3, 'result':'building_windows_non_float_processed'}
}
premise='Ba < 0.335, which indicates that the glass has a low barium content, suggesting it is less likely to be a type of glass \
that requires a high density or high refractive index, such as certain types of optical glass, and Mg > 2.78, which suggests \
that the glass may be designed for improved chemical durability and resistance to weathering. Magnesium is not typically present \
in high amounts in standard soda-lime glass, which is commonly used in windows. Those glass of above attributes are \
likely to be non-float processed.'
print(get_selection_prompt(
    desc=desc,
    role=role,
    options=opt_text,
    premise=premise
))

In [ ]:
response = '{C} \
The option A Refractive Index: Suggests "building_windows_non_float_processed" with 73% probability if RI < 1.517. \
However, this is less relevant since the glass is unlikely to be float processed. The option B Calcium: \
Indicates a 50% chance of "building_windows_float_processed" if Ca > 8.235, but float processing is unlikely given the conditions. \
The option C Aluminum: Suggests "building_windows_non_float_processed" with 70% probability if Al > 1.42. \
This aligns well with the instances memtioned with the question, making it a strong candidate. \
Option D Potassium: Indicates a 70% chance of being "tableware" if K < 0.03. This is a plausible option given the characteristics. \
Conclusion: Option C (Aluminum) is the best choice, given its high probability and alignment with non-float processed glass.'
status,res = get_selection_result('ABCDEFGHIJKLMNOPQRSTUVWXYZ', response)
print(status)
print(res[0])
print(res[-1])

In [42]:
# 安装 pandas and pyarrow:
!pip install pandas pyarrow

Defaulting to user installation because normal site-packages is not writeable
